<a href="https://colab.research.google.com/github/anmol-sinha-coder/DEmoClassi/blob/master/Age_Gender_Race_Emotion_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/G_Drive')

In [ ]:
! git clone https://github.com/anmol-sinha-coder/DEmoClassi.git
! cp -ra DEmoClassi/{vision_utils,emotion_detection,multitask_rag,'setup.py'} ./
! pip install tensorboardX pytorch-ignite pillow
! unzip /content/G_Drive/MyDrive/ADNN/facial-expression-recognition-challenge.zip -d .
! tar -xzvf /content/G_Drive/MyDrive/ADNN/UTKFace/UTKFace.tar.gz -C .
! tar -xzvf fer2013.tar.gz
! cp /content/G_Drive/MyDrive/ADNN/cv2_gpu/cv2.cpython-36m-x86_64-linux-gnu.so .

In [ ]:
import torch
import torchvision.transforms as transforms

from vision_utils.custom_torch_utils import load_model
from vision_utils.custom_architectures import SepConvModelMT, SepConvModel, initialize_model

from emotion_detection.evaluate import evaluate_model as eval_fer
from emotion_detection.fer_data_utils import *
from emotion_detection.train import run_fer

from multitask_rag.train import run_utk
from multitask_rag.utk_data_utils import get_utk_dataloader
from multitask_rag.evaluate import evaluate_model as eval_utk
from multitask_rag.utk_data_utils import display_examples_utk

import pandas as pd
import glob
import os
import random
import cv2
cv2.__version__

## Fer2013 dataset
Fer2013 is a kaggle dataset which consists of a set of 48x48 grayscale images representing the following facial expressions : 
* 0 : Angry
* 1 : Disgust
* 2 : Fear 
* 3 : Happy 
* 4 : Sad 
* 5 : Surprise 
* 6 : Neutral

In [ ]:
path_fer = './fer2013/fer2013.csv'
df_fer2013 = pd.read_csv(path_fer)

In [ ]:
display_examples_fer(df_fer2013, 0)

In [ ]:
display_examples_fer(df_fer2013, 1)

In [ ]:
display_examples_fer(df_fer2013, 2)

In [ ]:
display_examples_fer(df_fer2013, 3)

In [ ]:
display_examples_fer(df_fer2013, 4)

In [ ]:
display_examples_fer(df_fer2013, 5)

In [ ]:
display_examples_fer(df_fer2013, 6)

## UTKFace dataset
This is a dataset of cropped face images for the task of predicting the age, gender and race of a person.<br>

**Age :** A number between 0 and 101 (representing the age of the person)<br>

**Gender :**
* 0 : Male
* 1 : Female

**Race :**
* 0 : White
* 1 : Black
* 2 : Asian
* 3 : Indian
* 4 : Other


In [ ]:
path_utk = './UTKFace/'

In [ ]:
display_examples_utk(path_utk, 'gender', 0)

In [ ]:
display_examples_utk(path_utk, 'gender', 1)

In [ ]:
display_examples_utk(path_utk, 'race', 0)

In [ ]:
display_examples_utk(path_utk, 'race', 1)

In [ ]:
display_examples_utk(path_utk, 'age', 10)

# Training

Now that we have the data ready, let's move to the funniest part : model training!
As I have two separate datasets (`Fer2013` for emotion detection and `UTKFace` for gender-race-age prediction) we'll 
have to train two separate models. For each of the two tasks I tested 3 different architectures : 
* A CNN based on Depthwise Separable Convolution
* Finetuning a pretrained Resnet50
* Finetuning a pretrained VGG19

<hr size=10 color=black>

## Training emotion detector
### a. Depthwise Separable Convolution model
First we need to create DataLoader objects which are handy Pytorch objects for yielding batches of data during training.
Basically, what the following code does is : 
* read the csv file and convert the raw pixels into numpy arrays
* Apply some pre-processing operations : 
    * Histogram equalization ([see here for more information](https://en.wikipedia.org/wiki/Histogram_equalization))
    * Add a channel dimension so that the image becomes 48x48x1 instead of 48x48
    * Convert the numpy array to a pytorch tensor 

In [ ]:
DATA_DIR = "./fer2013/fer2013.csv" # path to the csv file
BATCH_SIZE = 256 # size of batches 
train_flag = 'Training' #`Usage` column in the csv file represents the usage of the data : train or validation or test
val_flag = 'PublicTest'

# The transformations to apply
data_transforms = transforms.Compose([
    HistEq(), # Apply histogram equalization
    AddChannel(), # Add channel dimension to be able to apply convolutions
    transforms.ToTensor()
])

train_dataloader = get_fer_dataloader(BATCH_SIZE, DATA_DIR, train_flag, data_transforms=data_transforms)
validation_dataloader = get_fer_dataloader(BATCH_SIZE, DATA_DIR, val_flag, data_transforms=data_transforms)

my_data_loaders = {
    'train': train_dataloader,
    'valid': validation_dataloader
}

In [ ]:
my_model = SepConvModel()  # 
my_optimizer = torch.optim.Adam(my_model.parameters(), lr=1e-3)

In [ ]:
backup_path = '/content/G_Drive/MyDrive/ADNN/AgeGenderClassification/Training_Records/'
os.makedirs(backup_path, exist_ok=True)  # create the directory if it doesn't exist
checkpoint = '/content/checkpoints/sep_conv'  # folder where to save checkpoints during training

In [ ]:
# Evaluation of model
run_fer(model=my_model,
        optimizer=my_optimizer,
        epochs=300,
        log_interval=1,
        dataloaders=my_data_loaders,
        dirname=checkpoint,
        n_saved=1,
        log_dir=None,
        launch_tensorboard=True,
        patience=50,
        resume_model=None,
        resume_optimizer=None,
        backup_step=5,
        backup_path=backup_path,
        n_epochs_freeze=0,
        n_cycle=None)